In [4]:
# pyarrow might be needed to read the data
!python -m pip install pyarrow

     |████████████████████████████████| 21.9 MB 4.3 MB/s eta 0:00:01


In [2]:
import numpy as np
import pandas as pd
from lc_classifier.features import ZTFLightcurvePreprocessor, ZTFFeatureExtractor

# Reading the sample data

To calculate the model features we will need:
- Detections: Alerts from the ZTF stream, with 5 $\sigma$ difference between the template and the current image.
- Non Detections: Previous measurements of the object but with less than 5 $\sigma$ magnitudes.
- Object Information: Aggregated information by ALeRCE.

In [5]:
detections = pd.read_parquet('data/detections_sample.parquet')
non_detections = pd.read_parquet('data/non_detections_sample.parquet')
object_information = pd.read_parquet('data/object_info_sample.parquet')

In [6]:
detections.head()

,fid,isdiffpos,sigmapsf,field,fwhm,dec,magpsf,rcid,ra,sky,rb,ssmagnr,distpsnr3,sgscore2,maggaiabright,distpsnr2,distpsnr1,maggaia,exptime,drb,sgscore3,neargaia,sgscore1,mjd,corrected,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext,dubious,has_stamp
oid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaemke,1,-1.0,0.098972,396,2.910000,0.802001,17.957479,61,358.760453,-0.130846,0.910000,-999.0,18.236210,0.026667,-999.0,12.874929,0.089459,15.460238,30.0,0.989089,0.441208,0.061395,0.99875,58700.461042,True,16.478576,0.008541,0.025349,False,True
ZTF17aaaemke,1,-1.0,0.075092,396,3.850000,0.801930,17.806704,61,358.760434,0.125953,0.891429,-999.0,18.109360,0.026667,-999.0,12.938040,0.304864,15.460238,30.0,0.999953,0.441208,0.274704,0.99875,58754.290857,True,16.520813,100.000000,0.022974,False,True
ZTF17aaaemke,1,-1.0,0.093225,396,3.160000,0.801997,17.616571,61,358.760472,0.457313,0.940000,-999.0,18.180468,0.026667,-999.0,12.938669,0.031614,15.460238,30.0,0.999997,0.441208,0.005324,0.99875,58763.377998,True,16.586325,0.024660,0.036094,False,True
ZTF17aaaemke,1,-1.0,0.146396,396,1.544062,0.801997,18.686962,61,358.760405,-0.294222,0.891429,-999.0,18.350967,0.026667,-999.0,12.729602,0.262691,15.460238,30.0,0.999442,0.441208,0.233289,0.99875,58718.413889,True,16.350400,100.000000,0.017018,False,True
ZTF17aaaemke,1,-1.0,0.091417,396,3.670000,0.801892,17.775181,61,358.760452,0.110218,0.616667,-999.0,17.966200,0.026667,-999.0,13.063952,0.409953,15.460238,0.0,0.000000,0.441208,0.386295,0.99875,58372.401910,True,16.530643,0.014738,0.029054,False,True


In [7]:
non_detections.head()

,parent_candid,jd,fid,pid,diffmaglim,pdiffimfilename,programpi,programid,candid,isdiffpos,tblid,nid,rcid,field,xpos,ypos,ra,dec,magpsf,sigmapsf,chipsf,magap,sigmagap,distnr,magnr,sigmagnr,chinr,sharpnr,sky,magdiff,fwhm,classtar,mindtoedge,magfromlim,seeratio,aimage,bimage,aimagerat,bimagerat,elong,nneg,nbad,rb,ssdistnr,ssmagnr,ssnamenr,sumrat,magapbig,sigmagapbig,ranr,decnr,scorr,magzpsci,magzpsciunc,magzpscirms,clrcoeff,clrcounc,rbversion,mjd
oid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaemke,618401906115010002,2.458353e+06,1,598377030615,20.065100,/ztf/archive/sci/2018/0822/376991/ztf_20180822...,Kulkarni,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58352.377037
ZTF17aaaemke,636353206115010001,2.458361e+06,1,606403590615,19.109699,/ztf/archive/sci/2018/0830/403600/ztf_20180830...,Kulkarni,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58360.403600
ZTF17aaaemke,618401906115010002,2.458364e+06,1,609312890615,20.057800,/ztf/archive/sci/2018/0902/312859/ztf_20180902...,Kulkarni,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58363.312894
ZTF17aaaemke,618401906115010002,2.458367e+06,1,612400500615,20.791800,/ztf/archive/sci/2018/0905/400509/ztf_20180905...,Kulkarni,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58366.400509
ZTF17aaaemke,618401906115010002,2.458370e+06,1,615325740615,20.812700,/ztf/archive/sci/2018/0908/325729/ztf_20180908...,Kulkarni,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58369.325741


In [8]:
object_information.head()

,ndethist,ncovhist,mjdstarthist,mjdendhist,meanra,meandec,sigmara,sigmadec,firstmjd,lastmjd,deltamjd,step_id_corr,nearZTF,nearPS1,stellar,corrected,ndet,ndubious,g-r_max,g-r_max_corr,g-r_mean,g-r_mean_corr
oid,,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaemke,135.0,670.0,58302.481944,59107.311562,358.760440,0.801990,0.000086,0.000060,58372.401910,59107.311562,734.909653,corr_bulk_0.0.1,True,False,True,True,88,0,1.209530,0.828800,0.531433,0.952658
ZTF17aaafyya,968.0,1471.0,58101.201285,59107.366215,17.787271,61.129036,0.000128,0.000066,58314.467755,59107.366215,792.898461,corr_bulk_0.0.1,True,True,True,True,375,3,0.757556,0.824260,0.635323,0.916122
ZTF17aaageae,280.0,783.0,58351.390069,59107.310625,8.686962,0.905769,0.000107,0.000067,58355.371991,59107.310625,751.938634,corr_bulk_0.0.1,True,True,True,True,217,0,0.168999,-0.030895,0.063114,0.207873
ZTF17aaaivsr,232.0,669.0,58101.410752,58981.216250,129.397864,9.654056,0.000060,0.000064,58375.516100,58981.216250,605.700151,corr_bulk_0.0.1,True,True,True,True,79,0,0.663141,0.630097,0.823536,0.559042
ZTF17aaaizej,332.0,428.0,58101.419444,58987.201377,141.224907,4.263769,0.000071,0.000050,58472.420324,58987.201377,514.781053,corr_bulk_0.0.1,True,True,True,True,100,0,-0.603085,-0.105530,0.113668,0.197381


# Calculating features

## Light curve preprocessing

In [11]:
preprocessor = ZTFLightcurvePreprocessor(stream=False)

detections = preprocessor.preprocess(detections, objects=object_information)
non_detections = preprocessor.rename_columns_non_detections(non_detections)

## Feature computation

In [15]:
feature_extractor = ZTFFeatureExtractor(bands=(1, 2), stream=False)

In [16]:
# We use the compute_features method
features = feature_extractor.compute_features(
    detections = detections,
    non_detections = non_detections)

 /home/ignacio/Projects/turbo-fats/turbofats/features/structure_function.py:88: RuntimeWarning:invalid value encountered in sqrt
 /home/ignacio/Projects/turbo-fats/turbofats/features/structure_function.py:97: RuntimeWarning:invalid value encountered in greater
 /home/ignacio/Projects/turbo-fats/turbofats/features/structure_function.py:88: RuntimeWarning:invalid value encountered in sqrt
 /home/ignacio/Projects/turbo-fats/turbofats/features/structure_function.py:97: RuntimeWarning:invalid value encountered in greater


In [20]:
# The result is a dataframe with the features
features

,Amplitude_1,Amplitude_2,AndersonDarling_1,AndersonDarling_2,Autocor_length_1,Autocor_length_2,Beyond1Std_1,Beyond1Std_2,Con_1,Con_2,Eta_e_1,Eta_e_2,ExcessVar_1,ExcessVar_2,GP_DRW_sigma_1,GP_DRW_sigma_2,GP_DRW_tau_1,GP_DRW_tau_2,Gskew_1,Gskew_2,Harmonics_mag_1_1,Harmonics_mag_1_2,Harmonics_mag_2_1,Harmonics_mag_2_2,Harmonics_mag_3_1,Harmonics_mag_3_2,Harmonics_mag_4_1,Harmonics_mag_4_2,Harmonics_mag_5_1,Harmonics_mag_5_2,Harmonics_mag_6_1,Harmonics_mag_6_2,Harmonics_mag_7_1,Harmonics_mag_7_2,Harmonics_mse_1,Harmonics_mse_2,Harmonics_phase_2_1,Harmonics_phase_2_2,Harmonics_phase_3_1,Harmonics_phase_3_2,Harmonics_phase_4_1,Harmonics_phase_4_2,Harmonics_phase_5_1,Harmonics_phase_5_2,Harmonics_phase_6_1,Harmonics_phase_6_2,Harmonics_phase_7_1,Harmonics_phase_7_2,IAR_phi_1,IAR_phi_2,LinearTrend_1,...,delta_mjd_fid_2,delta_period_1,delta_period_2,dmag_first_det_fid_1,dmag_first_det_fid_2,dmag_non_det_fid_1,dmag_non_det_fid_2,first_mag_1,first_mag_2,g-W2,g-W3,g-r_max,g-r_max_corr,g-r_mean,g-r_mean_corr,gal_b,gal_l,iqr_1,iqr_2,last_diffmaglim_before_fid_1,last_diffmaglim_before_fid_2,last_mjd_before_fid_1,last_mjd_before_fid_2,max_diffmaglim_after_fid_1,max_diffmaglim_after_fid_2,max_diffmaglim_before_fid_1,max_diffmaglim_before_fid_2,mean_mag_1,mean_mag_2,median_diffmaglim_after_fid_1,median_diffmaglim_after_fid_2,median_diffmaglim_before_fid_1,median_diffmaglim_before_fid_2,min_mag_1,min_mag_2,n_det_1,n_det_2,n_neg_1,n_neg_2,n_non_det_after_fid_1,n_non_det_after_fid_2,n_non_det_before_fid_1,n_non_det_before_fid_2,n_pos_1,n_pos_2,positive_fraction_1,positive_fraction_2,r-W2,r-W3,rb,sgscore1
oid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaemke,0.319945,0.277661,0.999998,0.999997,1.0,1.0,0.450000,0.363636,0.026316,0.0,0.067383,0.961166,0.000098,0.000154,0.027930,0.037457,0.021441,2.314798,-0.212015,-0.021045,0.541003,0.215071,0.054880,0.046276,0.522126,0.036496,0.483313,0.070920,0.143119,0.033470,0.338062,0.033562,0.192606,0.025595,2.612810e-03,1.995081e-03,0.423656,2.978978,3.331772,0.081824,6.016861,0.175531,0.960724,2.490062,2.644497,3.374112,5.153334,6.026703,4.230063e-13,6.562760e-01,0.000036,...,673.119849,0.000000,0.000000,4.223757,4.287914,4.512560,4.928362,16.530643,15.294985,3.10461,3.83761,0.673079,0.828800,0.502556,0.981895,-58.949304,94.752755,0.145523,0.370046,20.754400,19.582899,58372.287130,58431.186701,20.858801,20.700300,20.812700,20.459200,16.491376,15.509481,20.200001,19.968800,20.592699,20.179701,16.080139,15.251339,40.0,33.0,35.0,21.0,105.0,103.0,7.0,7.0,5.0,12.0,0.125000,0.363636,2.18798,2.92098,0.880000,0.998750
ZTF17aaafyya,0.373089,0.347666,1.000000,1.000000,1.0,1.0,0.404580,0.391753,0.000000,0.0,0.434366,0.001489,0.000298,0.000318,0.072054,0.067931,0.031425,0.083388,-0.133123,-0.028876,0.306503,0.294391,0.079725,0.083529,0.007974,0.013033,0.014693,0.010249,0.015882,0.004615,0.020052,0.022164,0.007097,0.013753,4.787496e-03,2.952426e-03,0.012544,0.018752,3.932469,4.674011,3.487734,4.178343,5.760150,3.385512,5.384526,1.312428,2.436749,2.178558,4.230063e-13,9.846708e-06,-0.000035,...,716.060625,0.000000,0.000000,5.088251,5.957441,5.153382,6.492919,15.258249,14.876960,NaN,NaN,0.757556,0.851637,0.786739,0.883384,-1.651650,125.309980,0.535644,0.526623,20.346500,20.834400,58316.429306,58368.344109,21.011299,20.883200,20.346500,20.834400,15.563753,14.680369,19.912001,20.096550,20.346500,20.834400,15.193118,14.341481,131.0,97.0,79.0,54.0,89.0,82.0,1.0,1.0,52.0,43.0,0.396947,0.443299,NaN,NaN,0.787857,0.996250
ZTF17aaageae,0.219709,0.338611,1.000000,1.000000,1.0,1.0,0.340000,0.105263,0.000000,0.0,0.020667,0.021095,0.000152,0.000160,0.030861,0.032353,0.479765,0.287250,0.091381,0.146700,0.289628,3.759873,0.077103,3.301431,0.174003,2.724665,0.036499,1.913007,0.123736,1.211113,0.023999,0.576385,0.047166,0.207938,2.612009e-04,6.504885e-03,3.872717,3.240022,4.346646,0.159277,1.999258,3.379438,2.941436,0.354118,0.306801,3.646671,1.326475,0.869292,1.380123e-01,2.